# Logic Graph Examples

Create interactive graphs of logical formulas with inference edges.

**Default:** `build_logic_graph` uses **forward-chaining** - starting from axioms and applying inference rules to derive new statements.

**Alternative:** `build_backward_logic_graph` uses **backward-chaining** - generating all possible formulas and finding relationships to axioms.

In [ ]:
# Example 1: Modus Ponens - x, (x → y) ⊢ y
from calculus_ratiocinator import Var, Implies, build_logic_graph, export_to_html

x = Var("x")
y = Var("y")
axioms = [x, Implies(x, y)]

# Build graph using forward-chaining (default)
g = build_logic_graph(["x", "y"], axioms, max_iterations=10)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check if y is entailed (it should be via modus ponens)
if str(y) in g.nodes:
    print(f"✓ y is entailed: {g.nodes[str(y)]['entailed']}")

# Export to interactive HTML (automatically saved in graphs/ folder)
export_to_html(g, "modus_ponens.html")

🔵 Starting with 2 axioms
🔵 Iteration 1: Generated 4 new formulas
🔵 Iteration 2: Generated 3 new formulas
🔵 Iteration 3: Generated 1 new formulas
🔵 Iteration 4: Generated 0 new formulas
✓ Fixed point reached at iteration 4
✓ Final knowledge base: 10 formulas, 10 nodes, 12 edges
Graph has 10 nodes and 12 edges
✓ y is entailed: True
Interactive graph saved to graphs\modus_ponens.html


In [8]:
# Example 2: Multiple inference rules in one graph
from calculus_ratiocinator import Var, Implies, build_logic_graph, export_to_html

p = Var("p")
q = Var("q")
r = Var("r")

# Axioms: p, (p → q), (q → r)
axioms = [p, Implies(p, q), Implies(q, r)]

g = build_logic_graph(["p", "q", "r"], axioms, max_iterations=3)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Show what was derived by generation
print("\n📋 Derived formulas by generation:")
for gen in range(4):
    gen_nodes = [n for n, d in g.nodes(data=True) if d.get('generation') == gen]
    if gen_nodes:
        print(f"  Gen {gen}: {', '.join(gen_nodes)}")

export_to_html(g, "complex.html")

🔵 Starting with 3 axioms
🔵 Iteration 1: Generated 7 new formulas
🔵 Iteration 2: Generated 6 new formulas
🔵 Iteration 3: Generated 5 new formulas
✓ Final knowledge base: 21 formulas, 21 nodes, 29 edges
Graph has 21 nodes and 29 edges

📋 Derived formulas by generation:
  Gen 0: p, (p → q), (q → r)
  Gen 1: q, ((q → r) ∧ (q → r)), ((q → r) ∧ (p → q)), ((q → r) ∧ p), ((p → q) ∧ (p → q)), ((p → q) ∧ p), (p ∧ p)
  Gen 2: r, ((q → r) ∧ q), (p ∧ q), (p ∧ (p → q)), (q ∧ q), (q ∧ (p → q))
  Gen 3: ((q → r) ∧ r), (p ∧ r), (q ∧ r), (r ∧ r), (r ∧ (p → q))
Interactive graph saved to graphs\complex.html


In [9]:
# Example 3: Conjunction elimination
from calculus_ratiocinator import Var, And, build_logic_graph, export_to_html

a = Var("a")
b = Var("b")

# Start with a conjunction axiom
axioms = [And(a, b)]

g = build_logic_graph(["a", "b"], axioms=axioms, max_iterations=2)

print(f"Total formulas: {g.number_of_nodes()}")

# Count by category
axiom_count = sum(1 for _, d in g.nodes(data=True) if d.get('is_axiom'))
entailed_count = sum(1 for _, d in g.nodes(data=True) if d.get('entailed'))
print(f"Axioms: {axiom_count}")
print(f"Entailed formulas: {entailed_count}")

# Show derivations
print("\n📋 Derived formulas by generation:")
for gen in range(3):
    gen_nodes = [n for n, d in g.nodes(data=True) if d.get('generation') == gen]
    if gen_nodes:
        print(f"  Gen {gen}: {', '.join(gen_nodes)}")

export_to_html(g, "conjunction.html")

🔵 Starting with 1 axioms
🔵 Iteration 1: Generated 2 new formulas
🔵 Iteration 2: Generated 2 new formulas
✓ Final knowledge base: 5 formulas, 5 nodes, 4 edges
Total formulas: 5
Axioms: 1
Entailed formulas: 5

📋 Derived formulas by generation:
  Gen 0: (a ∧ b)
  Gen 1: a, b
  Gen 2: (a ∧ a), (b ∧ b)
Interactive graph saved to graphs\conjunction.html


# Backward-Chaining Alternative

The examples above use **forward-chaining** (the default): start from axioms and apply inference rules to generate new entailed statements.

Below we demonstrate **backward-chaining**: generate all possible formulas up to a depth, then find relationships to axioms.

In [15]:
# Example 1: Modus Ponens - x, (x → y) ⊢ y
from calculus_ratiocinator import Var, Implies, build_backward_logic_graph, export_to_html

x = Var("x")
y = Var("y")
axioms = [x, Implies(x, y)]

# Build graph using forward-chaining (default)
g = build_backward_logic_graph(["x", "y"], axioms, max_depth=2, max_nodes=200)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check if y is entailed (it should be via modus ponens)
if str(y) in g.nodes:
    print(f"✓ y is entailed: {g.nodes[str(y)]['entailed']}")

# Export to interactive HTML (automatically saved in graphs/ folder)
export_to_html(g, "modus_ponens.html")

Graph has 200 nodes and 314 edges
✓ y is entailed: True
Interactive graph saved to graphs\modus_ponens.html


In [10]:
# Example 4: Backward-chaining approach
from calculus_ratiocinator import Var, Implies, build_backward_logic_graph, export_to_html

p = Var("p")
q = Var("q")
r = Var("r")

# Start with axioms: p, (p → q), (q → r)
axioms = [p, Implies(p, q), Implies(q, r)]

# Build graph by enumerating all formulas then finding relationships
g_backward = build_backward_logic_graph(["p", "q", "r"], axioms, max_depth=2, max_nodes=200)

print(f"\nGraph has {g_backward.number_of_nodes()} nodes and {g_backward.number_of_edges()} edges")

# Count by category
axiom_count = sum(1 for _, d in g_backward.nodes(data=True) if d.get('is_axiom'))
entailed_count = sum(1 for _, d in g_backward.nodes(data=True) if d.get('entailed'))
tautologies = sum(1 for _, d in g_backward.nodes(data=True) if d.get('tautology'))
print(f"Axioms: {axiom_count}")
print(f"Entailed: {entailed_count}")
print(f"Tautologies: {tautologies}")

export_to_html(g_backward, "backward_chaining.html")


Graph has 200 nodes and 290 edges
Axioms: 3
Entailed: 200
Tautologies: 34
Interactive graph saved to graphs\backward_chaining.html


## Understanding the Graph

**Node colors:**
- 🟠 Orange: Axiom formulas (starting points)
- 🟢 Light Green: Tautologies (always true)
- 🔵 Sky Blue: Entailed from axioms
- 🔴 Red: Not entailed (false under axioms)

**Edge colors (inference rules):**
- Dark Gray-Blue: General entailment
- Hot Pink: Modus Ponens (MP)
- Deep Purple: Modus Tollens (MT)
- Amber: Disjunctive Syllogism (DS)
- Cyan: Hypothetical Syllogism (HS)
- Indigo: Conjunction Elimination Left (∧E-L)
- Brown: Conjunction Elimination Right (∧E-R)

**Interactive features:**
- Drag nodes to rearrange
- Hover over nodes/edges for details
- Scroll to zoom
- Use navigation buttons in bottom-right

**Smart Sampling:**
- Perfect for exploring depth 3-4 without overwhelming the browser!
- Set `max_nodes` to limit graph size
- Prioritizes: axioms → entailed → tautologies → false statements (simpler first)